In [48]:
import pandas as pd
import numpy as np
from IPython.display import display

In [49]:
data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/가계, 노동 미시 버전 사회계정행렬(국산) RAS 적용.xlsx')


In [67]:
def sam_analysis():
    print('=======================================')

    print()
    print('사회계정행렬 선택: ')
    print('1. 거시 사회계정행렬 \n 2. 거시 사회계정행렬(국산) \n 3. 미시 사회계정행렬(산업 세분화) \n 4. 미시 사회계정행렬(산업 세분화: 국산) \n 5. 미시 사회계정행렬(최종) \n 6. 미시 사회계정행렬(최종: 국산) \n 7. 가계, 노동 분할 미시 사회계정행렬(최종:국산)')
    print()
    i = int(input('승수 분석을 진행할 SAM의 종류를 선택해주세요. : '))
    
    
    if i == 1:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM.xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[6:])
    elif i == 2:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
         
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/거시 SAM(국산).xlsx')
        print('데이터 불러오기 완료')
        print('========================================')
        name = list(data.columns[6:])
    elif i == 3:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬 RAS 적용.xlsx')
        name = list(data.columns[26:])
    elif i == 4:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(국산) RAS 적용.xlsx')
        name = list(data.columns[26:])
    
    elif i == 5:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종) RAS 적용.xlsx')        
        name = list(data.columns[27:])
    elif i == 6:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        # window 
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/미시 사회계정행렬(최종 국산) RAS 적용.xlsx')
        name = list(data.columns[27:])
        
    
    elif i == 7:
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/가계, 노동 미시 버전 사회계정행렬(국산) RAS 적용.xlsx')
        
        # window
        data = pd.read_excel('/Users/dongyokim/main/jupyter/2022:Summer/프로젝트/RAS 전처리 SAM/승수분석용 데이터/가계, 노동 미시 버전 사회계정행렬(국산) RAS 적용.xlsx')
        name = list(data.columns[32:])
    else:
        print('번호를 잘못 입력했습니다. 함수를 다시 실행해주세요. ')
        print('========================================')
    
    # untitled:0 삭제
    del name[0]
    
    # 총계 삭제
    del name[-1]
    data.rename(columns={'Unnamed: 0':'구분'},inplace=True)
    data.set_index('구분', inplace=True)
    data.fillna(0, inplace=True)
    data.replace(' ', 0, inplace=True)
    data = data.astype('float')
    
    
    print()
    
    print('데이터 전처리 및 불러오기 완료')
    print('========================================')
    
    
    s11 = data.loc[:'정부', :'정부'].values.copy()

    s12 = data.loc[:'정부', '고정자본형성':'수입'].values.copy()
    
    s21 = data.loc['고정자본형성':'수입', :'정부'].values.copy()
    
    s22 = data.loc['고정자본형성':'수입', '고정자본형성':'수입'].values.copy()
    
    
    y1_r = data.loc['총계', :'정부'].values.reshape(1, -1).copy()
    
    y2_r = data.loc['총계', '고정자본형성':'수입'].values.reshape(1, -1).copy()
        

    y1_c = data.loc[:'정부', '총계'].values.reshape(-1, 1).copy()
    y2_c = data.loc['고정자본형성':'수입', '총계'].values.reshape(-1, 1).copy()
    
    
    # 투입계수 도출 
    
    h11 = s11 / y1_r
    h12 = s12 / y2_r
    h21 = s21 / y1_r
    h22 = s22 / y2_r
    
        
    # 승수행렬
    SAM_multiplier = np.linalg.inv(np.identity(h11.shape[0])-h11)   
    print()
    print('승수 행렬 : ')
    print(SAM_multiplier)
    print()
    
    print(name)
    print()
    added_name = input("값을 추가하고 싶은 계정 이름을 입력해주세요: ")
    
    name_index = name.index(added_name)
    
    print('=======================')
    added_value = float(input('추가하실 금액을 입력해주세요: '))
    
    standard_value = y2_r[0][name_index].copy()
    
    y2_r[0][name_index] = y2_r[0][name_index] + added_value
    
    
    print("{}({})항목에 + {} = {}".format(added_name, standard_value,added_value, y2_r[0][name_index]))
    
    result_vector = np.dot(np.dot(SAM_multiplier, h12), y2_r.transpose()).transpose()    

    update_metrix = h11 * result_vector    
    
    
    data.loc[:'정부', :'정부'] = update_metrix

    data.drop(columns=['총계'], index=['총계'], inplace=True)
    
    
    print()
    print('승수분석을 완료하였습니다. ')
    print('=======================')
    display(data)
    
    return SAM_multiplier, data

In [89]:
sam_multiplier, data = sam_analysis()


사회계정행렬 선택: 
1. 거시 사회계정행렬 
 2. 거시 사회계정행렬(국산) 
 3. 미시 사회계정행렬(산업 세분화) 
 4. 미시 사회계정행렬(산업 세분화: 국산) 
 5. 미시 사회계정행렬(최종) 
 6. 미시 사회계정행렬(최종: 국산) 
 7. 가계, 노동 분할 미시 사회계정행렬(최종:국산)

승수 분석을 진행할 SAM의 종류를 선택해주세요. : 7

데이터 전처리 및 불러오기 완료

승수 행렬 : 
[[1.07932980e+00 7.98238294e-04 3.89491364e-02 ... 5.46924437e-02
  7.13534918e-03 2.90267290e-02]
 [1.46017764e-03 1.00005081e+00 3.38663626e-03 ... 1.88825290e-03
  3.02454637e-04 1.23039091e-03]
 [6.30214885e-01 1.57049470e-02 1.66207742e+00 ... 7.76172424e-01
  1.14077987e-01 4.64071306e-01]
 ...
 [2.83719968e-01 9.32173486e-03 2.36976779e-01 ... 1.29683817e+00
  8.67262178e-02 3.52803816e-01]
 [4.28633560e-01 1.04560933e-02 2.75584422e-01 ... 3.42621964e-01
  1.06776001e+00 2.75648943e-01]
 [1.05366682e-01 2.57031636e-03 6.77441499e-02 ... 8.42233155e-02
  2.62476716e-01 1.06776001e+00]]

['고정자본형성', 'SIT 고정자본형성', '간접세', '법인세', '소득세', '관세', '수출', '수입']

값을 추가하고 싶은 계정 이름을 입력해주세요: SIT 고정자본형성
추가하실 금액을 입력해주세요: 1369.3
SIT 고정자본형성(40175.24151130001)항목에 + 1369.3 

,농림어업,광업,제조업,도매 및 소매업,"전기, 가스, 증기 및 수도사업",건설업,운수업,숙박 및 음식점업,정보 및 통신업,금융 및 보험업,...,기업,정부,고정자본형성,SIT 고정자본형성,간접세,법인세,소득세,관세,수출,수입
구분,,,,,,,,,,,,,,,,,,,,,
농림어업,3394.683376,0.134472,28282.394733,45.016948,9.748999,312.065359,19.188279,8212.601850,17.944353,34.326979,...,0.000000,0.000000,326.204123,0.000000,0.000000,0.000,0.000000,0.000,1046.290200,0.000
광업,0.528530,0.077057,3617.670648,0.000000,540.065132,221.735077,0.000000,2.234226,0.000000,0.000000,...,0.000000,0.000000,1912.429521,0.000000,0.000000,0.000,0.000000,0.000,123.835920,0.000
제조업,14458.256886,21.109446,539842.160791,11087.888494,11855.095127,92755.757011,31337.411458,49618.800140,5371.563921,3833.066546,...,0.000000,0.000000,144207.302557,0.000000,0.000000,0.000,0.000000,0.000,626754.286552,0.000
도매 및 소매업,1151.146913,1.050841,35123.740902,4047.825380,642.987212,4633.032932,1545.877191,7631.145803,4543.237939,538.645354,...,0.000000,0.000000,7800.530749,0.000000,0.000000,0.000,0.000000,0.000,20929.141874,0.000
"전기, 가스, 증기 및 수도사업",741.987840,3.296513,32753.076955,2233.153026,14841.115417,964.020834,2311.367560,4697.964554,1147.709056,1329.428718,...,0.000000,6418.539950,0.000000,0.000000,0.000000,0.000,0.000000,0.000,1043.839590,0.000
건설업,79.935382,0.609338,1255.859189,210.264789,597.027875,96.342439,231.200562,370.986006,253.627423,103.124900,...,0.000000,0.000000,259315.407007,0.000000,0.000000,0.000,0.000000,0.000,218.694093,0.000
운수업,622.374304,19.610628,41694.835238,12488.253335,1672.290358,2288.689790,9592.003844,1074.506468,1653.473073,814.936702,...,0.000000,0.000000,587.368833,0.000000,0.000000,0.000,0.000000,0.000,33821.328244,0.000
숙박 및 음식점업,427.177960,2.683328,9086.785555,6319.127629,770.898778,944.134655,1680.628942,1527.924869,1563.094648,3921.607308,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,9634.183042,0.000
정보 및 통신업,158.848204,0.475144,3381.211834,5465.731292,326.621425,363.880340,1289.026080,1188.172526,9781.819836,6203.100454,...,0.000000,0.000000,16217.552763,0.000000,0.000000,0.000,0.000000,0.000,7811.449305,0.000


In [90]:
data.to_excel('SCN1_1_7.xlsx')